In [27]:
?apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91

Object `apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91` not found.


In [28]:
def get_dep(ticker):
 url = "https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?limit=120&apikey=67824182044bfc7088c8b3ee21824590".format(ticker=ticker)

 response = requests.request("GET", url)
 data = json.loads(response.text)

 return data[0]['depreciationAndAmortization']

In [29]:
pip install wacc

In [30]:
import pandas as pd
import requests
import numpy as np
import json
from scipy.stats.mstats import gmean

In [31]:
 url = "https://financialmodelingprep.com/api/v3/discounted-cash-flow/AAPL?apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91"

 response = requests.request("GET", url)
 data = json.loads(response.text)

In [32]:
data

[{'symbol': 'AAPL',
  'date': '2024-05-18',
  'dcf': 135.81272747667722,
  'Stock Price': 189.87}]

In [33]:
 url = "https://financialmodelingprep.com/api/v3/balance-sheet-statement/AAPL?period=annual&apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91"

 response = requests.request("GET", url)
 balance_sheet = json.loads(response.text)
 bs_try = balance_sheet[0]

In [34]:
 url = " https://financialmodelingprep.com/api/v3/cash-flow-statement/AAPL?period=annual&apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91"
 response = requests.request("GET", url)
 cashflow_statement = json.loads(response.text)
 cf_try = cashflow_statement[0]

In [35]:
 url = " https://financialmodelingprep.com/api/v3/income-statement/AAPL?period=annual&apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91"
 response = requests.request("GET", url)
 income_statement = json.loads(response.text)
 is_try = income_statement[0]

In [36]:
 url = "https://financialmodelingprep.com/api/v3/cash-flow-statement-growth/AAPL?period=annual&apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91"
 response = requests.request("GET", url)
 growth = json.loads(response.text)
 growth_try = growth[0]

In [37]:
 url = " https://financialmodelingprep.com/api/v4/shares_float?symbol=AAPL&apikey=NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91"
 response = requests.request("GET", url)
 shares = json.loads(response.text)
 shares[0]["outstandingShares"]


15334100000

In [38]:
shares

[{'symbol': 'AAPL',
  'date': '2024-05-18 05:29:35',
  'freeFloat': 99.9083,
  'floatShares': 15320038630,
  'outstandingShares': 15334100000,
  'source': 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm'}]

In [39]:
def calculate_dcf(cash_flow_base, growth_rate, discount_rate, projection_years, terminal_growth_rate):
    """
    Calculate the present value of future cash flows and terminal value for DCF.

    Parameters:
    - cash_flow_base (float): Base year operating cash flow.
    - growth_rate (float): Annual growth rate of cash flows.
    - discount_rate (float): Annual discount rate.
    - projection_years (int): Number of years for cash flow projection.
    - terminal_growth_rate (float): Growth rate to estimate the terminal value.

    Returns:
    - float: The DCF value of the cash flows and terminal value.
    """
    future_cash_flows = []
    for year in range(1, projection_years + 1):
        future_cash_flows.append(cash_flow_base * ((1 + growth_rate) ** year))

    # Calculate the terminal value
    terminal_value = future_cash_flows[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)

    # Discount cash flows and terminal value
    dcf_value = 0
    for year, cash_flow in enumerate(future_cash_flows, start=1):
        dcf_value += cash_flow / ((1 + discount_rate) ** year)
    dcf_value += terminal_value / ((1 + discount_rate) ** projection_years)

    return dcf_value

# Example usage:
cash_flow_base = cf_try['operatingCashFlow']
growth_rate = growth_try['growthFreeCashFlow']
discount_rate = 0.10  # Hypothetical discount rate of 10%
projection_years = 5  # Forecasting 5 years ahead
terminal_growth_rate = 0.02  # Long-term growth rate of 2%

dcf_value = calculate_dcf(cash_flow_base, growth_rate, discount_rate, projection_years, terminal_growth_rate)
print(f"The DCF value is: ${dcf_value:.2f}")


The DCF value is: $807866984558.62


In [40]:
growth_try['growthFreeCashFlow']

-0.10641314393905404

In [41]:
# Constants for DCF calculation
discount_rate = 0.06  # WACC
terminal_growth_rate = 0.025  # Long-term growth rate, assumed to be 2.5%
projection_years = 5  # Number of years to project

# Initial values from cash flow statement
initial_fcf = cf_try['freeCashFlow']
net_debt = bs_try['totalLiabilities'] - bs_try['totalStockholdersEquity']

# Calculating Future Free Cash Flows using growth rates
fcfs = [initial_fcf * (1 + growth_try['growthFreeCashFlow'])**i for i in range(1, projection_years + 1)]

# Calculating Terminal Value
terminal_value = fcfs[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)

# Discounting FCFs and Terminal Value back to present value
discounted_fcfs = [fcf / (1 + discount_rate)**i for i, fcf in enumerate(fcfs, start=1)]
discounted_terminal_value = terminal_value / (1 + discount_rate)**projection_years

# Calculating Enterprise Value
enterprise_value = sum(discounted_fcfs) + discounted_terminal_value

# Calculating Equity Value
equity_value = enterprise_value - net_debt

# Determining Intrinsic Share Price
shares_outstanding =  shares[0]["outstandingShares"] # Example: 16,325.819 million shares outstanding (a fictional value for the example)
intrinsic_share_price = equity_value / shares_outstanding

print(f"The intrinsic share price of AAPL is ${intrinsic_share_price:.2f} based on the DCF model.")


The intrinsic share price of AAPL is $86.11 based on the DCF model.


In [42]:
import requests

# API Key and Company Symbol
api_key = 'NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91'
symbol = 'AAPL'

# URLs for API Requests
cash_flow_url = f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?limit=120&apikey={api_key}'
company_profile_url = f'https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={api_key}'
discounted_cash_flow_url = f'https://financialmodelingprep.com/api/v3/discounted-cash-flow/{symbol}?apikey={api_key}'

def present_value(future_value, wacc, years):
    """Calculate the present value of future cash flows."""
    return future_value / ((1 + wacc) ** years)

def fetch_data(url):
    """Helper function to fetch data from the FMP API."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

# Fetch the cash flow data, company profile, and DCF information
cash_flows = fetch_data(cash_flow_url)
profile = fetch_data(company_profile_url)[0]
dcf_info = fetch_data(discounted_cash_flow_url)[0]

# Extract WACC, latest cash flow, and FCF growth rate from API
wacc = 0.1
last_cash_flow = cash_flows[0]['netCashProvidedByOperatingActivities']
fcf_growth_rate = 0.09 # Fetching the growth rate from the DCF endpoint

years = 10  # Number of years for projection
total_pv = 0

# Calculate present value of future cash flows using the FCF growth rate
for i in range(1, years + 1):
    projected_cf = last_cash_flow * ((1 + fcf_growth_rate) ** i)
    pv = present_value(projected_cf, wacc, i)
    total_pv += pv

# Calculate terminal value and its present value
terminal_value = (last_cash_flow * (1 + fcf_growth_rate) ** years) / (wacc - fcf_growth_rate)
terminal_pv = present_value(terminal_value, wacc, years)

total_pv += terminal_pv

print(f'The DCF value of {symbol} is ${total_pv:.2f}')

The DCF value of AAPL is $11141132565006.81


In [44]:
import requests

# API Key and Company Symbol
api_key = 'NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91'
symbol = 'AAPL'

# URLs for API Requests
cash_flow_url = f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?limit=120&apikey={api_key}'
company_profile_url = f'https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={api_key}'
discounted_cash_flow_url = f'https://financialmodelingprep.com/api/v3/discounted-cash-flow/{symbol}?apikey={api_key}'

def present_value(future_value, wacc, years):
    """Calculate the present value of future cash flows."""
    return future_value / ((1 + wacc) ** years)

def fetch_data(url):
    """Helper function to fetch data from the FMP API."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

# Fetch the cash flow data, company profile, and DCF information
cash_flows = fetch_data(cash_flow_url)
profile = fetch_data(company_profile_url)[0]
dcf_info = fetch_data(discounted_cash_flow_url)[0]

# Check if WACC is available
wacc = profile.get('WACC')
if wacc is None:
    print("WACC data not available for this company.")
    wacc = 0.10  # Default WACC if not available; use an estimated or industry average WACC

last_cash_flow = cash_flows[0]['netCashProvidedByOperatingActivities']
fcf_growth_rate = dcf_info['dcfGrowthRate']  # Fetching the growth rate from the DCF endpoint

years = 10  # Number of years for projection
total_pv = 0

# Calculate present value of future cash flows using the FCF growth rate
for i in range(1, years + 1):
    projected_cf = last_cash_flow * ((1 + fcf_growth_rate) ** i)
    pv = present_value(projected_cf, wacc, i)
    total_pv += pv

# Calculate terminal value and its present value
terminal_value = (last_cash_flow * (1 + fcf_growth_rate) ** years) / (wacc - fcf_growth_rate)
terminal_pv = present_value(terminal_value, wacc, years)

total_pv += terminal_pv

print(f'The DCF value of {symbol} is ${total_pv:.2f}')


WACC data not available for this company.


KeyError: 'dcfGrowthRate'

In [45]:
pip install financetoolkit -U

In [46]:
from financetoolkit import Toolkit
import pandas as pd

# Load tickers from the uploaded CSV file
file_path = 'unique_sp500_tickers.csv'
tickers_df = pd.read_csv(file_path)
tickers = tickers_df['Ticker'].tolist()

# Initialize the Toolkit with the tickers and the API key
toolkit = Toolkit(tickers, api_key="NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91", start_date="1985-01-01")

# Get the Weighted Average Cost of Capital (WACC)
wacc_data = toolkit.models.get_weighted_average_cost_of_capital()

# Reset the index to flatten the DataFrame, using unnamed levels as fallback
wacc_data_reset = wacc_data.reset_index()

# Melt the DataFrame to create columns `Ticker`, `Finance_Name`, `Year`, and `Value`
wacc_melted = wacc_data_reset.melt(id_vars=[wacc_data_reset.columns[0], wacc_data_reset.columns[1]], var_name='Year', value_name='Value')

# Rename the columns
wacc_melted.rename(columns={wacc_data_reset.columns[0]: 'Ticker', wacc_data_reset.columns[1]: 'Finance_Name'}, inplace=True)

# Print the transformed DataFrame
print(wacc_melted.head())

# Save the output to a CSV file
output_file = 'formatted_wacc_data.csv'
wacc_melted.to_csv(output_file, index=False)



Obtaining financial statements:   0%|          | 0/3 [00:00<?, ?it/s]

No data found for the following tickers: ABK, ABS, ACAS, ACE, ANDV, ANR, AV, AYE, BCR, BJS, BMC, BS, CA, CBE, CCE, CEPH, CFC, CFN, CMCSK, COV, CTX, CVH, DJ, DPS, DWDP, EK, EMC, ESV, FDO, FNM, FRE, GAS, GENZ, GGP, GLK, GR, GRN, HAR, HCBK, HNZ, HPH, HRS, HSP, JDSU, JNY, JOYG, KG, KFT, KORS, KRFT, KSE, LDW, LEH, LLL, LO, LUK, MEE, MFE, MHS, MI, MIL, MOLX, NOVL, NVLS, NYX, OMX, PCLN, PCS, PETM, PGN, PTV, Q, QTRN, RDC, RSH, RX, SBL, SCG, SGP, SHLD, SIAL, SLR, SMS, SWY, TEG, TIE, TLAB, TRB, TSO, TYC, USL, WCG, WFR, WIN, WLP, WYN, XL, XTO, YHOO


Obtaining financial statements:   0%|          | 0/3 [03:19<?, ?it/s]


AttributeError: 'DataFrame' object has no attribute '_constructor_from_mgr'

In [47]:
# Save the output to a CSV file
output_file = 'formatted_wacc_data.csv'
wacc_melted.to_csv(output_file, index=False)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'wacc_melted' is not defined

In [ ]:
wacc_melted.shape

In [ ]:
from financetoolkit import Toolkit

toolkit = Toolkit(["AAPL", "TSLA"], api_key="NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91")

toolkit.models.get_weighted_average_cost_of_capital()

In [ ]:
from financetoolkit import Toolkit

toolkit = Toolkit(["AAPL", "TSLA"], api_key="NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91")

altman_z_score = toolkit.models.get_altman_z_score()

In [ ]:
pip install transformers

In [ ]:
from transformers import DistilBertTokenizer

# Cargar el tokenizador de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

# Frase a tokenizar
sentence = "This guide demonstrates practical techniques"

# Tokenizar la frase
tokens = tokenizer.encode(sentence, add_special_tokens=False)

print(tokens)
